# Extracting environment features from .jsonl cache files

## Scheme A: avg/p90/p10{bbox longevity, num bboxes, bbox speed, bbox size, ego speed, change in size of bboxes, intersection of bboxes}

In [1]:
import jsonlines
import numpy as np
from pathlib import Path
from tqdm import tqdm

from ad_config_search.pipeline_output_features import get_final_features
from ad_config_search.utils import ray_map, get_fn_without_ext, split_run

import pandas as pd

%load_ext autoreload
%autoreload 2

### get all jsonl with no repeat csvs

In [2]:
loc1 = list(set(Path("../../ad-config-search/12-02-results/").glob("**/*.jsonl")) - set(list(Path("../../ad-config-search/12-02-results/").glob("**/ray_results/**/*.jsonl"))))  # removing 4 folders which contain repeats
loc2 = list(Path("../../ad-config-search/12-03-v2/").glob("**/*.jsonl"))
loc3 = list(Path("../../ad-config-search/12-06-v8/").glob("**/*.jsonl"))
loc4 = list(Path("../../ad-config-search/12-06-v7/").glob("**/*.jsonl"))

In [90]:
all_jsonls = loc1+loc2+loc3+loc4

In [4]:
len(all_jsonls)

442854

In [91]:
len([jsonl for jsonl in tqdm(all_jsonls) if len(list(jsonl.parent.glob("*.csv"))) != 1])

100%|██████████| 442854/442854 [01:28<00:00, 5029.27it/s]


11599

In [92]:
all_jsonls = [jsonl for jsonl in tqdm(all_jsonls) if len(list(jsonl.parent.glob("*.csv"))) == 1]

100%|██████████| 442854/442854 [01:33<00:00, 4747.96it/s]


In [6]:
len(all_jsonls)

431255

In [7]:
import ray

In [65]:
ray.shutdown()
ray.init(_redis_password="234k1234jlk2j34lkj34kj3kfjkfj", _temp_dir="/tmp/melih_is_using_the_default_ray_dir")

2021-02-14 19:45:29,078	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '169.229.48.125',
 'raylet_ip_address': '169.229.48.125',
 'redis_address': '169.229.48.125:6379',
 'object_store_address': '/tmp/melih_is_using_the_default_ray_dir/session_2021-02-14_19-45-27_307226_111842/sockets/plasma_store',
 'raylet_socket_name': '/tmp/melih_is_using_the_default_ray_dir/session_2021-02-14_19-45-27_307226_111842/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/melih_is_using_the_default_ray_dir/session_2021-02-14_19-45-27_307226_111842',
 'metrics_export_port': 60012,
 'node_id': '30967162ac85edfb8b3ebc88dfa2c13cb69d797d'}

In [66]:
part_length_s = 4
all_rows = ray_map(all_jsonls, lambda jsonl: get_final_features(jsonl, part_length_s), 120)    

100%|██████████| 3593/3593 [03:34<00:00, 16.72it/s]


In [67]:
import itertools
all_rows = list(itertools.chain.from_iterable(all_rows))

In [78]:
features = pd.DataFrame(all_rows)
features

,avg_bbox_longevity,90p_bbox_longevity,10p_bbox_longevity,avg_num_bboxes,90p_num_bboxes,10p_num_bboxes,avg_bbox_size,90p_bbox_size,10p_bbox_size,avg_bbox_speed,...,pastC-D-model,pastC-res-h,pastC-res-w,pastC-D-conf,pastC-D-seq-pol,pastC-T-model,pastC-T-min-iou,pastC-T-max-age,pastC-T-ignore-len,pastC-T-every-nth-det
0,9.583333,14.5,4.0,2.875000,5.0,0.0,195177.031863,294325.400000,128192.240000,172.191762,...,efficientdet-d1,1280,1920,0.3,tail-aware,sort,0.2,1,1,5
1,13.176471,31.0,1.0,5.600000,7.0,5.0,63057.921310,108681.900000,39316.892857,65.749491,...,efficientdet-d1,1280,1920,0.3,tail-aware,sort,0.2,1,1,5
2,25.181818,40.0,9.0,6.925000,8.1,5.0,58437.529345,73279.713889,49661.671429,43.374975,...,efficientdet-d1,1280,1920,0.3,tail-aware,sort,0.2,1,1,5
3,18.950000,40.0,4.9,9.475000,11.0,8.0,58399.155960,84967.600000,34437.420000,49.116288,...,efficientdet-d1,1280,1920,0.3,tail-aware,sort,0.2,1,1,5
4,16.950000,38.0,1.0,8.921053,13.0,7.0,42498.144741,51407.385714,35070.315385,58.326323,...,efficientdet-d1,1280,1920,0.3,tail-aware,sort,0.2,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156270,3.086957,3.0,3.0,1.775000,5.0,0.0,6435.890385,10262.200000,4314.750000,14.080128,...,efficientdet-d6,1280,1920,0.3,eager,sort,0.5,3,1,3
2156271,3.526316,6.0,3.0,1.675000,3.0,1.0,4925.621622,6968.000000,3979.500000,53.155989,...,efficientdet-d6,1280,1920,0.3,eager,sort,0.5,3,1,3
2156272,3.216216,3.0,1.0,2.975000,4.1,1.0,6138.419167,15744.000000,2292.525000,24.860260,...,efficientdet-d6,1280,1920,0.3,eager,sort,0.5,3,1,3
2156273,3.895522,6.0,2.6,6.525000,8.0,5.0,4016.249345,6399.866667,2375.800000,47.554961,...,efficientdet-d6,1280,1920,0.3,eager,sort,0.5,3,1,3


In [79]:
features = features.drop(columns=["pastC-dataset", "pastC-run", "pastC-res-h", "pastC-res-w", "pastC-T-ignore-len", "pastC-T-model"])

In [82]:
for column in ['weather', 'location', 'time_of_day', 'scenario_name', 'pastC-D-model', 'pastC-D-conf', 'pastC-D-seq-pol', 'pastC-T-min-iou', 'pastC-T-max-age', 'pastC-T-every-nth-det']:
    features[column] = features[column].astype("category")

In [83]:
features.memory_usage(deep=True), features.memory_usage(deep=True).sum()

(Index                         128
 avg_bbox_longevity       17250200
 90p_bbox_longevity       17250200
 10p_bbox_longevity       17250200
 avg_num_bboxes           17250200
 90p_num_bboxes           17250200
 10p_num_bboxes           17250200
 avg_bbox_size            17250200
 90p_bbox_size            17250200
 10p_bbox_size            17250200
 avg_bbox_speed           17250200
 90p_bbox_speed           17250200
 10p_bbox_speed           17250200
 avg_ego_speed            17250200
 90p_ego_speed            17250200
 10p_ego_speed            17250200
 time_of_day               2156543
 weather                   2156417
 location                  2156563
 scenario_name             4391190
 pastC-D-model             2157172
 pastC-D-conf              2156475
 pastC-D-seq-pol           2156690
 pastC-T-min-iou           2156675
 pastC-T-max-age           2156667
 pastC-T-every-nth-det     2156529
 dtype: int64,
 282554049)

In [84]:
features.columns

Index(['avg_bbox_longevity', '90p_bbox_longevity', '10p_bbox_longevity',
       'avg_num_bboxes', '90p_num_bboxes', '10p_num_bboxes', 'avg_bbox_size',
       '90p_bbox_size', '10p_bbox_size', 'avg_bbox_speed', '90p_bbox_speed',
       '10p_bbox_speed', 'avg_ego_speed', '90p_ego_speed', '10p_ego_speed',
       'time_of_day', 'weather', 'location', 'scenario_name', 'pastC-D-model',
       'pastC-D-conf', 'pastC-D-seq-pol', 'pastC-T-min-iou', 'pastC-T-max-age',
       'pastC-T-every-nth-det'],
      dtype='object')

In [85]:
def shift_rows_back_one(df, scenario_column):
    
    def get_section_number(section_name):
        return int(section_name.split("-P")[-1].split("_")[0])
    
    total_slices = int(df[scenario_column][0].split("_")[-1])
    
    # get rid of last sections
    df_cut = df[df[scenario_column].map(lambda x: get_section_number(x) < total_slices - 1)]
    df_copy = df_cut.copy()
    
    def increment_section_number(section_name):
        first_split = section_name.split("-P")
        first_part = first_split[0]
        last_part = first_split[-1]
        last_part_split = last_part.split("_")
        new_section_number = int(last_part_split[0])+1
        assert total_slices == int(last_part_split[1]), section_name
        assert new_section_number < total_slices, section_name
        return first_part+"-P{}_{}".format(new_section_number, last_part_split[1])
    
    df_copy[scenario_column] = [increment_section_number(name) for name in tqdm(df_copy[scenario_column])]
    return df_copy

In [86]:
past_features = shift_rows_back_one(features, "scenario_name")

100%|██████████| 1725020/1725020 [00:03<00:00, 470683.59it/s]


In [87]:
def get_section_number(section_name):
    return int(section_name.split("-P")[-1].split("_")[0])

In [88]:
assert 0 not in [get_section_number(n) for n in tqdm(past_features["scenario_name"])]

100%|██████████| 1725020/1725020 [00:02<00:00, 766721.62it/s]


In [89]:
past_features.to_parquet("past_features_from_pipeline_{}s_chunks.pq".format(part_length_s), compression=None)